<div style ="border: blue solid 5px; font-family: Agency FB;	"><center>
<a href="#" ></a> 
    <h1> <font color="blue">Traitement base DIANE</font> </h1> 
<a href="#" ></a>     
</center>
</div>

Dans ce notebook nous traitons la base DIANE, base contenant les informations financières des entreprises.

## Table des matières

* [Importation de la base](#1-importation-de-la-base)
* [Uniformisation des unités](#2-uniformisation-des-unités)
* [Création de la variable de fusion avec ADEME](#3-variable-de-fusion-avec-ademe)
* [Sauvegarde](#4-sauvegarde)

## 1. Importation de la base

In [79]:
import os
import pandas as pd
import numpy as np

In [80]:
os.chdir("D:/01_ENSAI/Sem_2/Projet_Statistique/projet-risque-de-transition")

In [81]:
diane = pd.read_excel('bdd/diane/Adway/new_diane_clean.xlsx', na_values = ["n.a.","-", "", 'n.s.', 'WO'])

In [82]:
df = diane.copy()

In [83]:
[col for col in df if "Purchases (goods), raw mater. and oth." in col]

['Purchases (goods), raw mater. and oth.']

## 2. Uniformisation des unités

In [84]:
unite = pd.read_excel("bdd/diane/Diane - Import des unités.xlsx", na_values=["n.a.","-",""])

In [85]:
unite['Unite'] = unite['Unité No of companies in corporate group']


In [86]:
## Variables pour la segmentation:
list_var = ['Net turnover', 
           'Average number of employees',
           'Total assets']

df[list_var]

,Net turnover,Average number of employees,Total assets
0,1.064521e+08,NaN,6.128550e+07
1,1.309588e+08,516.0,8.242856e+07
2,1.523906e+08,NaN,9.670874e+07
3,2.841351e+08,1344.0,1.477634e+08
4,3.635242e+08,1319.0,2.021669e+08
...,...,...,...
2281,3.334152e+08,1126.0,1.368135e+08
2282,7.600771e+05,4442.0,1.276729e+05
2283,3.919630e+08,2926.0,6.873140e+08
2284,3.517448e+08,NaN,3.028745e+08


In [87]:
df = df.merge(unite[['siren', 'Unite']], how = 'left', on='siren', validate='m:m')

In [88]:
df.shape

(3872, 639)

In [89]:
print(df.duplicated(subset=['siren', 'rep_year']).sum())
df.drop_duplicates(subset=['siren', 'rep_year'], inplace=True)

2267


In [90]:
euro = df.Unite.value_counts().index[0]
th_euro = df.Unite.value_counts().index[1]
m_euro = df.Unite.value_counts().index[2]

In [91]:
var_cont = (df
.drop([col for col in df if ('stock' in col)], axis = 1)
.select_dtypes('number')
.columns)  

var_cont = list(df[var_cont].mean()[df[var_cont].mean()>10000].sort_values(ascending = False).index)

In [92]:
df[['Total assets', 'Net turnover']].describe()

,Total assets,Net turnover
count,1.605000e+03,1.600000e+03
mean,1.141672e+08,1.615665e+08
std,1.343279e+08,1.838949e+08
min,1.000000e+04,0.000000e+00
25%,5.918115e+06,3.481654e+06
50%,7.049346e+07,1.029331e+08
75%,1.682991e+08,2.458796e+08
max,9.057800e+08,9.709609e+08


In [93]:
## Conversion des unités
euro = df.Unite.value_counts().index[0]
th_euro = df.Unite.value_counts().index[1]
m_euro = df.Unite.value_counts().index[2]
cond1 = df.Unite == euro
cond2 = df.Unite == th_euro
cond3 = df.Unite == m_euro
cond = [cond1, cond2, cond3]
for col in var_cont:
   values = [
      df[col],
      df[col]*1000,
      df[col]*1000000
   ]
   df[col] = np.select(cond, values)

In [95]:
df[['Total assets', 'Net turnover']].describe()

,Total assets,Net turnover
count,1.605000e+03,1.600000e+03
mean,8.627550e+08,7.886305e+08
std,3.109829e+09,3.124990e+09
min,1.000000e+04,0.000000e+00
25%,7.303006e+07,1.034063e+08
50%,1.725402e+08,2.348785e+08
75%,4.642237e+08,5.319377e+08
max,6.195443e+10,6.136230e+10


In [96]:
df = df.rename(columns={'Total assets': 'TotalAsset',
            'Net turnover': 'NetTurnover'
            })

## 3. Variable de fusion avec ADEME

In [97]:
df['rep_year'] = df['rep_year'].apply(int)
df['siren'] = df['siren'].apply(int)
df['SIREN_YEAR'] = df['siren'].apply(str) + df['rep_year'].apply(str) 
df['SIREN_YEAR']

0       7773464122010
1       7773464122015
2       7773464122018
3       5620508562010
4       5620508562019
            ...      
3867    3188261872020
3868    3291983372020
3869    3879495302020
3870    3302676912020
3871    9675059672020
Name: SIREN_YEAR, Length: 1605, dtype: object

## 4. Sauvegarde

In [98]:
df.to_csv('bdd/diane/Finale/diane.csv', index= False)